In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('detected.csv')
df

,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy bear,hair drier,toothbrush
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,8,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,1,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0
1927,1,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0
1928,1,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
1929,1,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0


In [3]:
#columns = df.columns.to_list()
#scaler = StandardScaler()
#df = scaler.fit_transform(df.values)
#df = pd.DataFrame(df, columns=columns)
#df

In [4]:
#from pyod.models.ecod import ECOD
#from pyod.models.knn import KNN
from pyod.models.iforest import IForest

df_results_pred = pd.DataFrame()
df_results_scores = pd.DataFrame()

models_dict = {
    #'ECOD': ECOD(),
    #'KNN': KNN(),
    'IForest': IForest(),
}

for model_name, model in models_dict.items():
    print(f'Fitting {model_name}...')
    model.fit(df.values)
    predictions = model.predict(df.values)
    scores = model.decision_scores_
    df_results_pred[model_name+"_p"] = predictions
    df_results_scores[model_name+"_s"] = scores

Fitting IForest...


In [5]:
# get the number of outliers (where all models agree)
df_results_pred['sum'] = df_results_pred.sum(axis=1)
df_results_pred['outlier'] = df_results_pred['sum'] == len(models_dict)
df_results_pred

,IForest_p,sum,outlier
0,0,0,False
1,0,0,False
2,0,0,False
3,0,0,False
4,0,0,False
...,...,...,...
1926,0,0,False
1927,0,0,False
1928,0,0,False
1929,0,0,False


In [6]:
df_results_pred["sum"].value_counts()

sum
0    1742
1     189
Name: count, dtype: int64

In [7]:
# normalize df with min-max normalization (esclude label column)
df_results_scores = (df_results_scores - df_results_scores.min()) / (df_results_scores.max() - df_results_scores.min())
# fill NaN values with 0
df_results_scores = df_results_scores.fillna(0)
df_results_scores["average_score"] = df_results_scores.mean(axis=1)
df_results_scores["average_score"] = df_results_scores["average_score"].round(3)
df_results_scores

,IForest_s,average_score
0,0.000000,0.000
1,0.006713,0.007
2,0.000000,0.000
3,0.000000,0.000
4,0.000000,0.000
...,...,...
1926,0.025219,0.025
1927,0.025219,0.025
1928,0.082886,0.083
1929,0.025219,0.025


In [8]:
outliers_index = df_results_pred[df_results_pred['outlier'] == True].index
df_results_scores = df_results_scores.loc[outliers_index]
df_results_scores

,IForest_s,average_score
149,0.634282,0.634
150,0.634282,0.634
151,0.716136,0.716
180,0.676454,0.676
200,0.662717,0.663
...,...,...
1765,0.596680,0.597
1766,0.596680,0.597
1768,0.596680,0.597
1771,0.596680,0.597


In [9]:
# plot the average score for each label with plotly
fig = px.scatter(df_results_scores, x=df_results_scores.index,
                  y="average_score",
                  title="Average score for each label", size="average_score", size_max=20, opacity=0.5)
fig.show()

In [10]:
df.iloc[1539][df.iloc[1539]!=0]

person    1
chair     2
remote    3
book      8
Name: 1539, dtype: int64

In [11]:
# create movin average column of average_score
df_results_scores['mov_avg'] = df_results_scores['average_score'].rolling(50).mean()
df_results_scores

,IForest_s,average_score,mov_avg
149,0.634282,0.634,NaN
150,0.634282,0.634,NaN
151,0.716136,0.716,NaN
180,0.676454,0.676,NaN
200,0.662717,0.663,NaN
...,...,...,...
1765,0.596680,0.597,0.69322
1766,0.596680,0.597,0.69328
1768,0.596680,0.597,0.69118
1771,0.596680,0.597,0.68908


In [12]:
# plot the moving average score for each label with plotly using line chart
fig = px.line(df_results_scores.iloc[50:], x=df_results_scores.iloc[50:].index,
                  y="mov_avg",
                  title="Moving average score for each label")
fig.show()

In [13]:
# save model to pickle
import pickle
filename = 'iforest_model_detected.pkl'
with open(filename, 'wb') as file:
    pickle.dump(models_dict['IForest'], file)
#pickle.dump(models_dict['IForest'], open(filename, 'wb'))